# Python API for Boolean and multi-valued network specification

The module `colomoto.minibn` offers a simple programmatic way to define and modify Boolean and multi-valued networks.
The network can then be converted to bioLQM for analysis.

Functions can be specified either as a string, or using standard Python logic operators (`~` for NOT; `&` for AND; `|` for OR).

## Boolean networks

In [1]:
from colomoto.minibn import BooleanNetwork

In [2]:
f = BooleanNetwork()

An object of type `BooleanNetwork` extends standard Python dictionnaries to specify for each node (key) the corresponding Boolean function.

To use a programmatic specifiction of functions, the variables for nodes are declared as follows:

In [3]:
a, b, c = f.vars("a", "b", "c")

In [4]:
f[a] = ~b & c
f[b] = ~a
f[c] = 1

Boolean expression can be stored in variables to reuse in more complex expressions:

In [5]:
d = b & c
f[b] = d
f[c] = ~d

In [6]:
f

a <- !b&c
b <- b&c
c <- !(b&c)


A Boolean network can also be specified directly as follows:

In [7]:
g = BooleanNetwork({
    "a": "!b & c",
    "b": "a",
    "c": "1"
})

Functions for `g` can then be edited as a standard dictionnary, using either the programmatic specification of expression or giving their string representation.

In [8]:
g["b"] = "b&c"

In [9]:
b, c = g.vars("b", "c")
g[c] = ~(b&c)

In [10]:
g

a <- !b&c
b <- b&c
c <- !(b&c)


### Rewriting expressions

The method `rewrite` allows to substitute literals with Boolean expressions:

In [11]:
g.rewrite(b, {c: a|c})
g

a <- !b&c
b <- b&(a|c)
c <- !(b&c)


In [12]:
g.rewrite("c", {b:1})
g

a <- !b&c
b <- b&(a|c)
c <- !c


### Conversion to bioLQM

An object of type `BooleanNetwork` has a method `to_biolqm` to convert the object to a full bioLQM object and perform further analysis or convert to other tools.

In [13]:
import biolqm

In [14]:
lqm = f.to_biolqm()

In [15]:
from colomoto_jupyter import tabulate

In [16]:
tabulate(biolqm.fixpoints(lqm))

|    |   a |   b |   c |
|---:|----:|----:|----:|
|  0 |   1 |   0 |   1 |

## Multi-Valued networks

Specification of multi-valued networks essentially extends `BooleanNetwork` features with ability to refer to levels of nodes and specify an ordering for the evaluation of the discrete expression.

Levels are specified programmatically using the syntax `a/i` (represented as a string with `"a:i"`) where `a` is a node variable of the network and `i` an integer. This variable is true when the current level of `a` is greater or equal to `i`.

In [17]:
from colomoto.minibn import MultiValuedNetwork

In [18]:
mvf = MultiValuedNetwork()

In [19]:
a, b, c, d = mvf.vars("a", "b", "c", "d")

Discrete functions can be specified using dictionnary giving the condition to reach the different levels. With this specification, the function will evaluate to the highest level having a verified condition.

In [20]:
mvf[a] = {1: b/1, 
          2: b/1 & c}

A more fine-grained specification allow to define the precise ordering of evaluation (the last win):

In [21]:
mvf.append(b/2, a/2 | ~c)
mvf.append(b/1, a/2 & c)

Multi-valued networks can include pure Boolean node, and are specified as with `BooleanNetwork`.

In [22]:
mvf[c] = c & ~(a/2) # notice the parentheses to comply with operators priority

In [23]:
mvf

a:1 <- b:1
a:2 <- b:1&c
b:2 <- a:2|!c
b:1 <- a:2&c
c <- c&!a:2


Conditions on levels can also be specified using standard comparison operators:

In [24]:
mvg = MultiValuedNetwork()
a, b, c = mvg.vars("a", "b", "c")
mvg.append(a/1, (b<=2))
mvg.append(a/0, (b<2))
mvg.append(a/3, (b>=2))
mvg.append(a/4, (b>2))
mvg.append(a/2, (b==2))
mvg[c] = (b!=2)
mvg

a:1 <- !b:3
a:0 <- !b:2
a:3 <- b:2
a:4 <- b:3
a:2 <- b:2&!b:3
c <- !b:2|b:3


A multi-valued network can also be instanciated with its textual representation:

In [25]:
mvg = MultiValuedNetwork("""
a:1 <- b:1
a:2 <- b:1 & c
b:1 <- a:2
b:2 <- a:2 & !c
c <- 1
""")

### Rewriting expressions

The rewriting of expressions is extended to support node levels in the specification of substitutions:

In [26]:
a, b = mvg.vars("a", "b")
mvg.rewrite(b, {a/2: a/1}) # will rewrite all expressions targetting b
mvg

a:1 <- b:1
a:2 <- b:1&c
b:1 <- a:1
b:2 <- a:1&!c
c <- 1


In [27]:
mvg.rewrite(a/2, {b/1: b/2}) # will rewrite only expressions for a:2
mvg

a:1 <- b:1
a:2 <- b:2&c
b:1 <- a:1
b:2 <- a:1&!c
c <- 1


### Conversion to bioLQM

Similarly to `BooleanNetwork` objects, `MultiValuedNetwork` objects have a `to_biolqm` method to convert to bioLQM tool.

In [28]:
mvf

a:1 <- b:1
a:2 <- b:1&c
b:2 <- a:2|!c
b:1 <- a:2&c
c <- c&!a:2


In [29]:
lqm = mvf.to_biolqm()

In [30]:
tabulate(biolqm.fixpoints(lqm))

|    |   a |   b |   c |
|---:|----:|----:|----:|
|  0 |   0 |   0 |   1 |
|  1 |   1 |   2 |   0 |

## Import bioLQM models

The `biolqm.to_minibn` function imports any Boolean/Multi-valued network in the `minibn` interface, enabling programmatic edition of the model.

In [31]:
lqm = biolqm.load("http://ginsim.org/sites/default/files/ModelT2DM_0.zginml")

Downloading 'http://ginsim.org/sites/default/files/ModelT2DM_0.zginml'

In [32]:
f = biolqm.to_minibn(lqm)
f

AKT <- (!IRS_PIK3CA&!PTEN&CDK2&!PP2A)|(IRS_PIK3CA&!PTEN&!PP2A)
AMPK <- p53&!ATP
ATP <- Metabolism
CDK2 <- (!p21&mTORC1_S6K1&!MYC&E2F1)|(!p21&mTORC1_S6K1&MYC)
E2F1 <- (!GF&MYC&!pRB&E2F1)|(GF&!pRB&E2F1)
FOXO <- (!MAPK&!p16&!AKT&!AMPK&Metabolism)|(!MAPK&!p16&!AKT&AMPK)|(!MAPK&p16&!AKT)
G1_S <- !p21&CDK2&E2F1&Metabolism
GF <- GF
IRS_PIK3CA <- Insulin&!mTORC1_S6K1
Insulin <- Insulin
MAPK <- GF&!PP2A
MDM2 <- (!p16&!p53&AKT&!mTORC1_S6K1&!MYC&!E2F1)|(!p16&p53&!mTORC1_S6K1&!MYC&!E2F1)|(p16&!mTORC1_S6K1&!MYC&!E2F1)
MYC <- MAPK&!p53&mTORC1_S6K1&E2F1
Metabolism <- (!MAPK&!AKT&mTORC1_S6K1&PP1C)|(!MAPK&AKT&mTORC1_S6K1)|(MAPK&!AKT&PP1C)|(MAPK&AKT)
PP1C <- (!MAPK&AKT)|MAPK
PP2A <- !mTORC1_S6K1
PRC <- !AKT&MYC
PTEN <- p53&!AKT
Senescence <- (!p16&p21&mTORC1_S6K1)|p16
TSC <- !MAPK&!AKT&AMPK
Therapy <- Therapy
mTORC1_S6K1 <- !AMPK&!TSC
p16 <- MAPK&!p53&!E2F1&!PRC
p21 <- (!p53&!AKT&!MYC&FOXO)|(p53&!AKT&!MYC)
p53 <- !MDM2
pRB <- !CDK2


In [33]:
lqm_mv = biolqm.load("http://ginsim.org/sites/default/files/phageLambda4.zginml")

Downloading 'http://ginsim.org/sites/default/files/phageLambda4.zginml'

In [34]:
mv = biolqm.to_minibn(lqm_mv)
mv

CI:2 <- !Cro|(Cro&CII)
CII <- !CI:2&!Cro:3&N
Cro:2 <- !CI:2&Cro:3
Cro:3 <- !CI:2&!Cro:3
N <- !CI&!Cro:2
